# Model Analysis

In [3]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Number of GPUs Available:  1


In [5]:
from preprocess import *
from model import *
from utils import *
import random
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns


np.random.seed(2)
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, MaxPooling2D, UpSampling2D, Input, Concatenate, Conv2DTranspose, BatchNormalization,Reshape
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


from PIL import Image, ImageChops, ImageEnhance,ImageFilter, ImageOps
from io import BytesIO
import cv2
from scipy.fftpack import dct
from scipy import ndimage
from scipy import fftpack
import optuna


from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
import numpy as np
import glob
import random
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

def build_model(image_size=(128, 128)):
    filters =  32
    dropout_rate = 0.25

    model = Sequential()
    model.add(Conv2D(filters=filters, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(image_size[0], image_size[1], 3)))
    model.add(Conv2D(filters=filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(filters=2*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(filters=2*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(filters=4*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(filters=4*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.summary()

    return model

def train_model(model, train_generator, val_generator, batch_size, epochs):
    init_lr = 1e-4
   
    opt = Adam(learning_rate= init_lr, decay=init_lr / epochs)

    model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=0, mode='auto')
    checkpoint = tf.keras.callbacks.ModelCheckpoint('../models/model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)
    
    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=len(val_generator),
        callbacks=[early_stopping, checkpoint, tensorboard_callback]
    )
    return model

if __name__ == '__main__':
    real_images_path = '../data/dataset/data/CASIA2/Au/*.*'
    fake_images_path = '../data/dataset/data/CASIA2/Tp/*.*'
    image_size = (128, 128)

    batch_size = 32
    epochs = 50
    image_size = (128, 128)
    X = []
    y = []

    print("Cargando datos de imágenes reales...")
    for file_path in glob.glob(real_images_path):
        X.append(preparete_image_ela(file_path, image_size))
        y.append(0)

    random.shuffle(X)

    print("Cargando datos de imágenes falsificadas...")
    for file_path in glob.glob(fake_images_path):
        X.append(preparete_image_ela(file_path, image_size))
        y.append(1)

    X= np.array(X).reshape(-1, image_size[0], image_size[1], 3)
    y = to_categorical(y, num_classes = 2)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=5)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=5)

    train_generator = ImageDataGenerator().flow(X_train, y_train, batch_size=batch_size)
    val_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=batch_size)

    model = build_model(image_size)
    model = train_model(model, train_generator, val_generator, batch_size, epochs)

    # Calculamos la precisión del modelo con los datos de test
    score = model.evaluate(X_test, y_test)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Guardamos el modelo
    model.save('../models/model'+str(score[1])+'.h5')

Cargando datos de imágenes reales...
Cargando datos de imágenes falsificadas...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_13 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 64, 64, 32)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 64, 

: 

In [ ]:
'''import os
os.environ["PATH"] += os.pathsep + 'C:/Users/luisp/anaconda3/envs/test/Lib/site-packages/skimage/graphviz/'''
from keras.models import load_model

from keras.utils import plot_model

# Cargar el modelo entrenado
model = load_model('../models/best_model_0.9072.h5')

# Generar la representación gráfica del modelo
plot_model(model, to_file='arquitectura_del_modelo.png', show_shapes=True, show_layer_names=True)
